# <center> Hands-On : Minimax </center>
##  In this project, we want to implement **Minimax Algorithm** to play Checkers.

***
## What is Minimax Algorithm?
#### Mini-max algorithm is a recursive or backtracking algorithm which is used in decision-making and game theory. It provides an optimal move for the player assuming that opponent is also playing optimally.   [More information](https://www.javatpoint.com/mini-max-algorithm-in-ai)

***
## _traverseLeft() and _traverseRight():
#### These recursive functions are used to determine the moves that each Piece can take. 
### Parameters:
- **start**: The row number of the Piece
- **stop**: How far can a Piece go
- **step**: Defining that the Piece should go up or down(the RED player goes up and the WHITE player goes down)
- **color**: The color of the Piece
- **left** or **right**: The left or right cell of the piece
- **skipped**: A list of the Pieces that would be skipped after these moves<br>

#### General description:
In each function, we have a loop on all of the rows from **start** to **stop** and we determine whether we should go up or down with **step**.<br>Then we check to not going out of the board.(**if left < 0** or **if right > COLS**) <br> In every iteration, we check that the current cell is empty or not. Here we have 3 options:<br>
1. The current cell is empty<br>
2. The current cell is not empty and has a Piece that its color is the same as the initial Piece color, which means we can't move.<br>
3. The current cell is not empty and has a Piece that its color is different from the initial Piece color, so that means we can jump over it but notice that we don't know the place that we will land is empty or not, and in the next iteration we will find out.<br>
Then if we skipped from a Piece, we can recursively call the functions to check whether we can double jump or more.

In [ ]:
def _traverseLeft(self, start, stop, step, color, left, skipped=[]):
    moves = {}
    last = []
    for r in range(start, stop, step):
        if left < 0:
            break

        current = self.board[r][left]
        if current == 0:
            if skipped and not last:
                break
            elif  :
                moves[(r, left)] = last + skipped
            else:
                moves[(r, left)] = last

            if last:
                if step == -1:
                    row = max(r-3, 0)
                else:
                    row = min(r+3, ROWS)
                moves.update(self._traverseLeft(r+step, row, step, color, left-1,skipped=last))
                moves.update(self._traverseRight(r+step, row, step, color, left+1,skipped=last))
            break
        elif current.color == color:
            break
        else:
            last = [current]

        left -= 1

    return moves

def _traverseRight(self, start, stop, step, color, right, skipped=[]):
    moves = {}
    last = []
    for r in range(start, stop, step):
        if right >= COLS:
            break

        current = self.board[r][right]
        if current == 0:
            if skipped and not last:
                break
            elif skipped:
                moves[(r,right)] = last + skipped
            else:
                moves[(r, right)] = last

            if last:
                if step == -1:
                    row = max(r-3, 0)
                else:
                    row = min(r+3, ROWS)
                moves.update(self._traverseLeft(r+step, row, step, color, right-1,skipped=last))
                moves.update(self._traverseRight(r+step, row, step, color, right+1,skipped=last))
            break
        elif current.color == color:
            break
        else:
            last = [current]

        right += 1

    return moves

***
## getValidMoves()
#### This function should return the valid moves that each piece can do, considering the Piece's color and whether the Piece is a king or not.
Clarifying some pieces of the code:
- **$max(-1,  piece.row-3)$**: Means that we only see at most two rows above and also we check that we are not out of the board.
- **$min(8,  piece.row+3)$**: Means that we only see at most two rows below and also we check that we are not out of the board.

<br>**moves** is a dictionary that its keys are the destinitions and its values are the Pieces that are skipped.



In [ ]:
def getValidMoves(self, piece):
    moves = {}
    if piece.king == True:
        moves.update(self._traverseLeft(piece.row-1, max(-1, piece.row-3), -1, piece.color, piece.col-1))
        moves.update(self._traverseRight(piece.row-1, max(-1, piece.row-3), -1, piece.color, piece.col+1))
        moves.update(self._traverseLeft(piece.row+1, min(8, piece.row+3), 1, piece.color, piece.col-1))
        moves.update(self._traverseRight(piece.row+1, min(8, piece.row+3), 1, piece.color, piece.col+1))
    else:
        if piece.color == RED:
            moves.update(self._traverseLeft(piece.row-1, max(-1, piece.row-3), -1, piece.color, piece.col-1))
            moves.update(self._traverseRight(piece.row-1, max(-1, piece.row-3), -1, piece.color, piece.col+1))
        else:
            moves.update(self._traverseLeft(piece.row+1, min(8, piece.row+3), 1, piece.color, piece.col-1))
            moves.update(self._traverseRight(piece.row+1, min(8, piece.row+3), 1, piece.color, piece.col+1))

    return moves

## simulateMove()
#### In this function we update the board based on a move.<br>


## getAllMoves()
#### In this function we first get all of the pieces of a specific color, then for each Piece, we get the valid moves from the getValidMoves() function. After that, we simulate those moves on the table and create a board for each of those moves. Finally, we return all of these boards.

In [ ]:
def simulateMove(piece, move, board, skip):
    row = move[0]
    col = move[1]
    
    board.move(piece, row, col)
    
    if skip:
        board.remove(skip)
    return board

def getAllMoves(board, color):
    all_moves = []
    pieces = board.getAllPieces(color)
    
    for p in pieces:
        valid_moves = board.getValidMoves(p)
        for move, skipped in valid_moves.items():
            board_copy = deepcopy(board)
            piece_copy = board_copy.getPiece(p.row, p.col)
            new_move_board = simulateMove(piece_copy, move, board_copy, skipped)
            all_moves.append(new_move_board)
    return all_moves

## Evaluation Function
#### We can simply define the **Evaluation Function** as an estimation of the goodness of a position. In the other words, we want to give our state a value that can estimate its goodness.<br> The evaluation function is used here is: $ #RED players - #WHITE players + #RED_kings * 0.5 - #WITE_kings * 0.5 $ <br> Because at the end, the number of players defines the winner, so we take it into account. Also because the king pieces are stronger than normal pieces, we include this issue too.

***
## Minimax
#### In this function, we check whether we should maximize the utility or minimize it. Based on the color of the players we should do this. The max_player() and the min_player() functions will do this job.<br>In these functions, we recursively go through our game tree and we will obtain the best moves by maximizing utility for one player and minimizing utility for another player.<br><br> **Minimax Properties:**
- **Complete**: if a solution exists, minimax will find it. (if the decision tree is finite)
- **Optimal**: minimax is optimal if both opponents are playing optimally.
- **Time Complexity**: $O(b^m)$, where b is branching factor of the game-tree, and m is the maximum depth of the tree.(because we perform DFS)
- **Space Complexity**: $O(bm)$ (because we perform DFS)

In [ ]:
def minimax(position, depth, maxPlayer):
    if depth == 0:
        return position.evaluate(), position
    
    if maxPlayer == True:
        return max_player(position, depth)

    else:
        return min_player(position, depth)

def max_player(position, depth):
    if depth == 0:
        return position.evaluate(), position
    move = position
    moves = getAllMoves(position, WHITE)
    v_max = -math.inf
    for i in moves:
        eval_ = min_player(i, depth-1)[0]
        if v_max <= eval_:
            v_max = eval_
            move = i
    return v_max, move

def min_player(position, depth):
    if depth == 0:
        return position.evaluate(), position
    move = position
    moves = getAllMoves(position, RED)
    v_min = math.inf
    for i in moves:
        eval_ = max_player(i, depth-1)[0]
        if v_min >= eval_:
            v_min = eval_
            move = i
    return v_min, move

***
## Running the game
#### In the main function, we run the game. For each turn, we run the minimax algorithm considering its depth to find the best move.

In [ ]:
FPS = 60

WHITE_DEPTH = 5
RED_DEPTH = 5

WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Checkers')

if __name__ == '__main__':
    run = True
    clock = pygame.time.Clock()
    game = Game(WIN)
    turn = 0
    while run:
        clock.tick(FPS)

        if game.turn == WHITE:
            value, newBoard = minimax(game.getBoard(), WHITE_DEPTH, True)
            if game.getBoard() == newBoard:
                turn += 1
            game.aiMove(newBoard)

        if turn > 10 or turn <-10:
            pygame.time.delay(10000)
            if turn > 0:
                print('There is no move for WHITE player!')
            else:
                print('There is no move for RED player!')
            if game.getBoard().redLeft > game.getBoard().whiteLeft:
                print('The RED Player wins!')
            elif game.getBoard().redLeft < game.getBoard().whiteLeft:
                print('The WHITE Player wins!')
            else:
                print('Tie!')
            run = False
            
        if game.winner() != None:
            print(game.winner())
            run = False

        if game.turn == RED:
            value, newBoard = minimax(game.getBoard(), RED_DEPTH, False)
            if game.getBoard() == newBoard:
                turn -= 1
            game.aiMove(newBoard)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

        game.update()
        # pygame.time.delay(500)
    
    pygame.quit()

***
## Testing the algorithm for the depth of 1 for both players
#### We can see that the game runs very fast because we just see one move ahead for each player. But because we just see one depth of the game tree, the decisions that each player makes, are not good.

***
## Testing the algorithm for the depth of 2 and 5
#### The game runs slower than the first test because here we see more depth from the tree, hence we make better decisions. But because the depth of the players is not the same, the player that has a bigger depth, sees more moves and will get better results.

***
## Testing the algorithm for the depth of 5 for both players
#### Here the game runs very slower than the previous tests because now we see a lot more things, that will result in making a lot better decisions compared to previous tests. But the depth of both players is the same, so they are equally strong.

***
## Conclusion
#### The main advantage of minimax is that it performs a deep evaluation in the field of search space. But if we have a game that has a large branching factor, it gets really slow. Here we can do some optimization such as **alpha-beta pruning** to get better time efficiency. With **alpha-beta pruning**, we can throw away those states which are not necessary to expand. 